In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
region = 'AKCP_weekly'
folder = 'projects/alpod-412314/assets/' + region

In [5]:
image_list = ee.data.listImages(folder)
image_list = image_list.get('images')

In [6]:
print(image_list[0:5])

[{'type': 'IMAGE', 'name': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_19', 'id': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_19', 'updateTime': '2024-06-25T16:34:29.002148Z'}, {'type': 'IMAGE', 'name': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_22', 'id': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_22', 'updateTime': '2024-06-25T16:33:44.811683Z'}, {'type': 'IMAGE', 'name': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_27', 'id': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_27', 'updateTime': '2024-06-25T16:36:17.478348Z'}, {'type': 'IMAGE', 'name': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_30', 'id': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_30', 'updateTime': '2024-06-25T16:45:20.412980Z'}, {'type': 'IMAGE', 'name': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_38', 'id': 'projects/alpod-412314/assets/AKCP_weekly/03WWT_2016_38', 'updateTime': '2024-06-25T16:44:29.203329Z'}]


In [7]:
target_year = '2018'
target_week = '27'

In [8]:
target_imgs = []

for image in image_list:
    img_id = image['id']
    year = img_id.split('_')[-2]
    week = img_id.split('_')[-1]

    if week == target_week and year == target_year:
        img_ee = ee.Image(img_id)
        target_imgs.append(img_ee)

print(len(target_imgs))

8


In [9]:
target_mosaic = ee.ImageCollection(target_imgs).mosaic()

In [12]:
export_task = ee.batch.Export.image.toDrive(
    image=target_mosaic,
    description='test',
    folder='ewebb_s2',
    fileNamePrefix='test_mosaic',
    scale=10,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

export_task.start()

In [59]:
viz1 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 100,
    'palette': ['ff0000', '0000ff']

Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(target_mosaic, viz1, '.')
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…